In [4]:
from pyspark.sql import SparkSession

print("--- CREANDO RELACIONES (FOREIGN KEYS) ---")

# 1. Configuración Centralizada
DB_HOST = "sql_server"
DB_USER = "sa"
DB_PASS = "PasswordFuerte123!"
DRIVER_PKG = "com.microsoft.sqlserver:mssql-jdbc:11.2.0.jre8"

# 2. Iniciar Spark
spark = SparkSession.builder \
    .appName("Fix-FK") \
    .master("local[*]") \
    .config("spark.jars.packages", DRIVER_PKG) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# 3. Función de Ejecución SQL (Método Directo)
def ejecutar_sql(query, db):
    # Construimos la URL limpia
    url = f"jdbc:sqlserver://{DB_HOST}:1433;databaseName={db};encrypt=true;trustServerCertificate=true;"
    
    try:
        # --- MAGIA: Conexión Directa al Driver ---
        # En lugar de pedirle al "Gerente" (DriverManager) que busque un chofer,
        # llamamos al chofer (SQLServerDriver) directamente. Esto evita el error "No suitable driver".
        
        # A. Accedemos a la JVM de Spark
        jvm = spark.sparkContext._gateway.jvm
        
        # B. Instanciamos el Driver y las Propiedades
        driver = jvm.com.microsoft.sqlserver.jdbc.SQLServerDriver()
        props = jvm.java.util.Properties()
        
        # C. Configuramos credenciales en el objeto Properties
        props.setProperty("user", DB_USER)
        props.setProperty("password", DB_PASS)
        
        # D. Conectamos
        con = driver.connect(url, props)
        
        if con is None:
            print(f"   ❌ Error: El driver no aceptó la URL.")
            return

        # E. Ejecutamos la query
        stmt = con.createStatement()
        stmt.execute(query)
        print(f"   ✅ Relación creada: {query[:60]}...")
        
        stmt.close()
        con.close()
        
    except Exception as e:
        error_msg = str(e)
        if "already exists" in error_msg or "conflicts with" in error_msg:
            print(f"   ℹ️ Ya existía: {query[:40]}...")
        else:
            print(f"   ❌ Error crítico: {error_msg}")

# --- EJECUCIÓN ---

print("\n>>> 1. Conectando Tablas OLTP (Transaccional)...")
fk_oltp = [
    "ALTER TABLE dbo.CONTRATO ADD CONSTRAINT FK_Contrato_Cliente FOREIGN KEY(idCliente) REFERENCES dbo.CLIENTE (idCliente)",
    "ALTER TABLE dbo.CONTRATO ADD CONSTRAINT FK_Contrato_Habitacion FOREIGN KEY(idHabitacion) REFERENCES dbo.HABITACION (idHabitacion)",
    "ALTER TABLE dbo.CONTRATO ADD CONSTRAINT FK_Contrato_Propietario FOREIGN KEY(idPropietario) REFERENCES dbo.PROPIETARIO (idPropietario)",
    "ALTER TABLE dbo.PAGO ADD CONSTRAINT FK_Pago_Contrato FOREIGN KEY(idContrato) REFERENCES dbo.CONTRATO (idContrato)"
]
for q in fk_oltp: ejecutar_sql(q, "alquiler_habitacion")

print("\n>>> 2. Conectando Data Warehouse (Estrella)...")
fk_dw = [
    "ALTER TABLE dbo.FACT_PAGO ADD CONSTRAINT FK_Fact_Cliente FOREIGN KEY(idClienteDW) REFERENCES dbo.DIM_CLIENTE (idClienteDW)",
    "ALTER TABLE dbo.FACT_PAGO ADD CONSTRAINT FK_Fact_Habitacion FOREIGN KEY(idHabitacionDW) REFERENCES dbo.DIM_HABITACION (idHabitacionDW)",
    "ALTER TABLE dbo.FACT_PAGO ADD CONSTRAINT FK_Fact_Propietario FOREIGN KEY(idPropietarioDW) REFERENCES dbo.DIM_PROPIETARIO (idPropietarioDW)",
    "ALTER TABLE dbo.FACT_PAGO ADD CONSTRAINT FK_Fact_Tiempo FOREIGN KEY(idTiempoDW) REFERENCES dbo.DIM_TIEMPO (idTiempoDW)"
]
for q in fk_dw: ejecutar_sql(q, "DW_AlquilerHabitacion")

print("\n✨ ¡Listo! Ahora actualiza DBeaver para ver las líneas.")

--- CREANDO RELACIONES (FOREIGN KEYS) ---

>>> 1. Conectando Tablas OLTP (Transaccional)...
   ❌ Error crítico: An error occurred while calling o83.execute.
: com.microsoft.sqlserver.jdbc.SQLServerException: There is already an object named 'FK_Contrato_Cliente' in the database.
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDatabaseError(SQLServerException.java:265)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.getNextResult(SQLServerStatement.java:1673)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.doExecuteStatement(SQLServerStatement.java:907)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement$StmtExecCmd.doExecute(SQLServerStatement.java:802)
	at com.microsoft.sqlserver.jdbc.TDSCommand.execute(IOBuffer.java:7627)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.executeCommand(SQLServerConnection.java:3912)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement.executeCommand(SQLServerStatement.java:268)
	at com.microsoft.sqlserver.jdbc.SQLServerStatement